# Retriever and Chain with Langchain

In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("yolo.pdf")
docs = loader.load()
docs

[Document(metadata={'source': 'yolo.pdf', 'page': 0}, page_content='You Only Look Once:\nUniﬁed, Real-Time Object Detection\nJoseph Redmon\nUniversity of Washington\npjreddie@cs.washington.eduSantosh Divvala\nAllen Institute for Artiﬁcial Intelligence\nsantoshd@allenai.org\nRoss Girshick\nFacebook AI Research\nrbg@fb.comAli Farhadi\nUniversity of Washington\nali@cs.washington.edu\nAbstract\nWe present YOLO, a new approach to object detection.\nPrior work on object detection repurposes classiﬁers to per-\nform detection. Instead, we frame object detection as a re-\ngression problem to spatially separated bounding boxes and\nassociated class probabilities. A single neural network pre-\ndicts bounding boxes and class probabilities directly from\nfull images in one evaluation. Since the whole detection\npipeline is a single network, it can be optimized end-to-end\ndirectly on detection performance.\nOur uniﬁed architecture is extremely fast. Our base\nYOLO model processes images in real-ti

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]


[Document(metadata={'source': 'yolo.pdf', 'page': 0}, page_content='You Only Look Once:\nUniﬁed, Real-Time Object Detection\nJoseph Redmon\nUniversity of Washington\npjreddie@cs.washington.eduSantosh Divvala\nAllen Institute for Artiﬁcial Intelligence\nsantoshd@allenai.org\nRoss Girshick\nFacebook AI Research\nrbg@fb.comAli Farhadi\nUniversity of Washington\nali@cs.washington.edu\nAbstract\nWe present YOLO, a new approach to object detection.\nPrior work on object detection repurposes classiﬁers to per-\nform detection. Instead, we frame object detection as a re-\ngression problem to spatially separated bounding boxes and\nassociated class probabilities. A single neural network pre-\ndicts bounding boxes and class probabilities directly from\nfull images in one evaluation. Since the whole detection\npipeline is a single network, it can be optimized end-to-end\ndirectly on detection performance.\nOur uniﬁed architecture is extremely fast. Our base\nYOLO model processes images in real-ti

In [3]:
documents = text_splitter.split_documents(docs)

In [5]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents[:30], OpenAIEmbeddings())

In [6]:
query = "Yolo v7 features"
result = db.similarity_search(query)
result[0].page_content

'Third, YOLO learns generalizable representations of ob-\njects. When trained on natural images and tested on art-\nwork, YOLO outperforms top detection methods like DPM\nand R-CNN by a wide margin. Since YOLO is highly gen-\neralizable it is less likely to break down when applied to\nnew domains or unexpected input.\nAll of our training and testing code is open source and\navailable online at [removed for review]. A variety of pre-\ntrained models are also available to download.\n2. Uniﬁed Detection\nWe unify the separate components of object detection\ninto a single neural network. Our network uses features\nfrom the entire image to predict each bounding box. It\nalso predicts all bounding boxes for an image simultane-\nously. This means our network reasons globally about the\nfull image and all the objects in the image. The YOLO de-\nsign enables end-to-end training and real-time speeds while\nmaintaining high average precision.\nOur system divides the input image into a S×Sgrid. If

In [8]:
from langchain_community.llms import Ollama

# Ollama LLama2
llm = Ollama(model="llama2")
llm

Ollama()

In [9]:
# Design chatprompt template

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based on provided context only.
Think step by step before providing answer.
<context>
{context}
</context>
Question: {input}""")

In [10]:
# Chain instruction
# Create stuff document chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [11]:
retriever = db.as_retriever()

In [12]:
# Retrival chain

from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [14]:
response = retrieval_chain.invoke({"input": "Limitations of yolo"})

In [16]:
response['answer']

'According to the provided context, the limitations of YOLO (You Only Look Once) are:\n\n1. Spatial constraints: YOLO imposes strong spatial constraints on bounding box predictions, limiting the number of nearby objects that can be predicted.\n2. Limited generalization: While YOLO learns very general representations of objects, it performs poorly when generalizing from natural images to artwork on both the Picasso Dataset and the People-Art Dataset. Other detection methods, including DPM and R-CNN, perform better in this regard.'